In [ ]:
%load_ext autoreload
%autoreload 2

import json
import dill
import numpy as np
from pydrake.geometry import StartMeshcat

from iiwa_batter import PACKAGE_ROOT, CONTROL_TIMESTEP, PITCH_TIMESTEP, CONTACT_TIMESTEP
from iiwa_batter.robot_constraints.get_joint_constraints import JOINT_CONSTRAINTS
from iiwa_batter.physics import find_initial_velocity, PITCH_START_POSITION

from iiwa_batter.swing_optimization.full_trajectory import run_full_trajectory, setup_simulator, make_torque_trajectory

In [ ]:
meshcat = StartMeshcat()

In [ ]:
# Load control vector
with open("best_control_vector.dill", "rb") as f:
    best_control_vector, _, _ = dill.load(f)

simulator, station = setup_simulator(dt=CONTACT_TIMESTEP, meshcat=meshcat)
initial_velocity, time_of_flight = find_initial_velocity(90, [0, 0, 0.6])
trajectory_timesteps = np.arange(0, time_of_flight, CONTROL_TIMESTEP)
robot_constraints = JOINT_CONSTRAINTS["kr6r900"] 
torque_trajectory = make_torque_trajectory(best_control_vector, robot_constraints, trajectory_timesteps)

run_full_trajectory(meshcat, simulator, station, best_control_vector[:7], [PITCH_START_POSITION, initial_velocity], time_of_flight, robot_constraints, torque_trajectory)